# 3. Рекомендательная система на основе содержимого

Существует еще один тип рекомендательных систем, известный как рекомендатель на основе содержимого (контента). Этот тип рекомендательных систем использует описание элемента, чтобы рекомендовать следующий наиболее похожий продукт. Рекомендаторы, основанные на содержании, также дают «персонализированные» рекомендации. Основное различие между рекомендателем, основанным на корреляции, и рекомендателем, основанным на контенте, заключается в том, что первый учитывает «поведение пользователя», а второй — контент для формирования рекомендации. Рекомендатор на основе содержимого использует функции продукта или ключевые слова, используемые в описании, чтобы найти сходство между элементами. Построим такую систему.

In [1]:
# Импортируем библиотеки
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

Функция **linear_kernel** используется для вычисления линейного ядра между двумя переменными. 
Следует отметить, что "ядра" — это меры сходства, т. е **s(a, b) > s(a, c)** если объекты a и b считаются «более похожими», чем объекты **a** и **c**. Ядро также должно быть положительно полуопределенным.
Мы используем эту функцию, а не косинусную меру сходства (**cosine_similarities()**) потому, что она быстрее, а также потому, что мы используем **TF-IDF vectorization**, простое скалярное произведение даст нам ту же оценку косинусного сходства. 
Что такое **TF-IDF vector**? Мы не можем вычислить сходство между описаниями в таком виде, в каком они представлены в нашем датасете. Чтобы решить эту проблему, вычислим **Frequency-Inverse Document Frequency (TF-IDF)** для всех документов датасета.
**TF-IDF (от англ. TF — term frequency, IDF — inverse document frequency)** — статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса. Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции.

Результатом будет матрица, где каждое слово представляет столбец. 
**sklearn** решает эту задачу за пару строк кода с использованием объекта **TfidfVectorizer**:

In [2]:
# reading file
book_description = pd.read_csv('description.csv', encoding = 'latin-1')

In [3]:
# checking if we have the right data
book_description.head()

,book_id,name,description
0,4833.0,The Glass Castle,"A tender, moving tale of unconditional love in..."
1,590.0,"Night (The Night Trilogy, #1)","Born into a Jewish ghetto in Hungary, as a chi..."
2,4264.0,"Angela's Ashes (Frank McCourt, #1)",Imbued on every page with Frank McCourt's asto...
3,3361.0,"Eat, Pray, Love","A celebrated writer's irresistible, candid, an..."
4,4535.0,Into Thin Air: A Personal Account of the Mount...,A bank of clouds was assembling on the not-so-...


In [5]:
# Удаляем из текста стоп-слова
books_tfidf = TfidfVectorizer(stop_words='english')
# replace NaN with empty strings
book_description['description'] = book_description['description'].fillna('')
# Рассчитываем матрицу TF-IDF, требуемую для расчета косинусной меры подобия текстов
book_description_matrix = books_tfidf.fit_transform(book_description['description'])


In [7]:
# Оценим размерность получившейся матрицы
book_description_matrix.shape

(143, 4186)

Выведенный результат означает, что в базе для описания 143 книг используется 4186 слов. 

In [8]:
# рассчитаем косинусную меру подобия с использованием линейного ядра
cosine_similarity = linear_kernel(book_description_matrix, book_description_matrix)

In [9]:
# Получим попарные оценки сходства всех книг по сравнению с книгой, переданной по индексу, отсортировав их и получив топ-5
# здесь 2 - это индекс книги в наборе данных
similarity_scores = list(enumerate(cosine_similarity[2]))
similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
similarity_scores = similarity_scores[1:6]

# Получим индекс похожих книг
books_index = [i[0] for i in similarity_scores]

# Выведем топ-5 наиболее похожих книг, используя индексацию на основе целочисленного местоположения (iloc)
print (book_description['name'].iloc[books_index])


6                                 Running with Scissors 
29                            The Diary of a Young Girl 
116    It's St. Patrick's Day (Turtleback School & Li...
11     Persepolis: The Story of a Childhood (Persepol...
20     Maus I: A Survivor's Tale: My Father Bleeds Hi...
Name: name, dtype: object


Мы получили результат, свидетельствующий о том, что для нашей рекомендательной системы книга с индексом 2 аналогична книге с индексом 6. Давайте просмотрим аннотации книг и оценим, как хорошо работает наш рекомендатель.

Вот краткое описание книги N2 - «Пепел Анжелы»:
**«Когда я оглядываюсь назад на свое детство, я удивляюсь, как мне вообще удалось выжить. Это было, конечно, несчастное детство: счастливое детство вряд ли стоит вашего труда. Хуже обычного несчастного детства несчастное ирландское детство, и что еще хуже, несчастное ирландское католическое детство».**

А вот аннотация книги №6 - «Бег с ножницами»:
**«Правдивая история о детстве вне закона, где правила были неслыханными, рождественская елка не спала круглый год, валиум потреблялся как конфеты, а если дела становились скучными, мог развлечься электрошоковой терапией».**

Мы видим определенное сходство между кратким описанием произведений. Также обе книги относятся к жанру «Биографии и мемуары». Это показывает, что наша рекомендация достаточно хороша при всей ее простоте.